In [1]:
import pandas as pd
import re

def clean_city_prefix(text):
    """
    Удаляет всё ДО и включая 'г. Город' или 'г.Город'
    """
    if pd.isna(text):
        return text
    text = str(text).upper()
    text = re.sub(r'^.*?\bГ\.\s*\w+', '', text)
    return text.strip()

def remove_apartment_part(text):
    """
    Удаляет квартиры и дом:
    - 'кв', 'кв.', 'кв12', 'кв.12А'
    - 'к.' если после цифра
    - 'квартира' + номер
    - 'дом', 'д.'
    """
    if pd.isna(text):
        return text
    text = str(text)

    # Удаляем квартиру
    text = re.sub(r',?\s*\bкв\.?\s*\d+[А-Яа-я]?\b.*', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\bкв\.?\d+[А-Яа-я]?\b.*', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\s*\bк\.\s*\d+[А-Яа-я]?\b.*', '', text, flags=re.IGNORECASE)
    text = re.sub(r',?\s*\bквартира\s*\d+[А-Яа-я]?\b.*', '', text, flags=re.IGNORECASE)

    # Удаляем дом
    text = re.sub(r'\bдом\b\.?', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\bд\.\s*', '', text, flags=re.IGNORECASE)

    return text.strip()

def remove_address_keywords(text):
    """
    Удаляет слова: улица, ул, ул., микрорайон, мкр, мкр., переулок, проспект
    """
    if pd.isna(text):
        return text
    text = str(text)

    # Удаляем указанные слова
    text = re.sub(r'\b(улица|ул\.?|микрорайон|мкр\.?|переулок|проспект)\b', '', text, flags=re.IGNORECASE)

    return re.sub(r'\s+', ' ', text).strip()

# Загружаем файл
df = pd.read_excel("General_legal (2).xlsx")

# Удаляем пустые строки
original_column = 'legal_address'
df = df[df[original_column].notna() & (df[original_column].astype(str).str.strip() != '')]

# Обработка
df['cleaned_address'] = (
    df[original_column]
    .apply(clean_city_prefix)
    .apply(remove_apartment_part)
    .apply(remove_address_keywords)
)

# Сохраняем
df.to_excel("General_legal_cleaned.xlsx", index=False)

print("✅ Всё готово! Удалены квартиры, 'дом', 'г.', и уличные обозначения. Сохранено в 'General_legal_cleaned.xlsx'")


✅ Всё готово! Удалены квартиры, 'дом', 'г.', и уличные обозначения. Сохранено в 'General_legal_cleaned.xlsx'


In [2]:
import pandas as pd

# Загрузка файла
df = pd.read_excel("General_KT.xlsx")  # замените на ваш путь к файлу

# Заменяем "-" на пустую строку в столбце 1.dev_housing
df["1.dev_housing"] = df["1.dev_housing"].replace("-", "")

# Объединяем столбцы через запятую, исключая пустые
df["full_address"] = df[["1.dev_street_name", "1.dev_house", "1.dev_housing"]].apply(
    lambda row: ", ".join([str(val) for val in row if str(val).strip()]), axis=1
)

# Сохраняем результат
df.to_excel("general_kt_cleaned.xlsx", index=False)


In [3]:
import pandas as pd
from rapidfuzz import process, fuzz

# Загрузка
df1 = pd.read_excel("General_legal_cleaned.xlsx")  # с 'cleaned_address'
df2 = pd.read_excel("general_kt_cleaned.xlsx")     # с 'full_address'

# Очистка от NaN и ?-знаков
def normalize(text):
    if pd.isna(text):
        return ''
    text = str(text).upper()
    text = text.replace('?', '')  # убираем знаки вопроса
    text = ' '.join(text.split())  # удаляем лишние пробелы
    return text.strip()

df1['norm_address'] = df1['cleaned_address'].apply(normalize)
df2['norm_address'] = df2['full_address'].apply(normalize)

# Список адресов для поиска
target_list = df2['norm_address'].tolist()

# Порог совпадения
THRESHOLD = 80

# Поиск лучшего совпадения + скор
def find_match_with_score(addr):
    match = process.extractOne(addr, target_list, scorer=fuzz.partial_ratio)
    if match and match[1] >= THRESHOLD:
        return pd.Series({'matched_address': match[0], 'score': match[1]})
    return pd.Series({'matched_address': None, 'score': None})

# Применяем
matches = df1['norm_address'].apply(find_match_with_score)
df1 = pd.concat([df1, matches], axis=1)

# Объединяем с исходным df2
merged_df = df1.merge(df2, left_on='matched_address', right_on='norm_address', how='left', suffixes=('_1', '_2'))

# Сохраняем
merged_df.to_excel("fuzzy_merged_addresses.xlsx", index=False)

print("✅ Сравнение завершено. Результат сохранён в 'fuzzy_merged_addresses.xlsx'")


✅ Сравнение завершено. Результат сохранён в 'fuzzy_merged_addresses.xlsx'


In [2]:
import pandas as pd
from rapidfuzz import process, fuzz

# Загрузка данных
df_kt = pd.read_excel("general_kt_cleaned.xlsx")
df_legal = pd.read_excel("General_legal_cleaned.xlsx")

# Приводим device_town к нижнему регистру
df_kt['device_town_norm'] = df_kt['1.device_town'].astype(str).str.lower().str.strip()

# Функция: ищем по kato_level_0 → kato_level_3 в нижнем регистре
def match_device_town(town):
    for level in ['kato_level_0', 'kato_level_1', 'kato_level_2', 'kato_level_3']:
        level_values = df_legal[level].dropna().astype(str).str.lower().tolist()
        result = process.extractOne(town, level_values, scorer=fuzz.partial_ratio)
        if result and result[1] >= 85:
            return pd.Series({'matched_level': level, 'matched_value': result[0], 'score': result[1]})
    return pd.Series({'matched_level': None, 'matched_value': None, 'score': None})

# Применяем к каждому device_town
matches = df_kt['device_town_norm'].apply(match_device_town)
df_kt = pd.concat([df_kt, matches], axis=1)

# Сохраняем результат
df_kt.to_excel("device_town_matched_lower.xlsx", index=False)

print("✅ Всё сопоставлено в lower case. Результат: 'device_town_matched_lower.xlsx'")

✅ Всё сопоставлено в lower case. Результат: 'device_town_matched_lower.xlsx'
